## TODO

- train with train, val and test set (need to write a loop)
- instead of sampling from data, split data evenly into 7 chunks and train through them
- install tf 1.11 or 1.12, move to tf.keras and try other models
- try ensemble (stacking and bagging)

In [9]:
import sys
print(sys.executable)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
from skimage.data import imread
import os,cv2
print(os.listdir("data"))
from sklearn.model_selection import train_test_split


train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train.head(5)

y = train.food_type

/usr/bin/python
['train_images', 'train.csv', 'test_images', 'test.csv']


## Data Loading

In [10]:
seed = 220
x_train, x_val, y_train, y_val = train_test_split(train.index, 
                                                  train.food_type, 
                                                  test_size=0.15, 
                                                  random_state=seed, 
                                                  stratify=train.food_type)

X = train.iloc[x_val]

from tqdm import tqdm
train_dir = 'data/train_images'

images = []
train_types = []

train_files = [train_dir + '/' + filename for filename in X.filename]
print(len(train_files))


for filename in tqdm(train_files):
    if filename.endswith('jpg'):
        try:
            images.append(cv2.resize(cv2.imread(filename), (299,299), interpolation=cv2.INTER_CUBIC))
            train_types.append(train[train_dir + '/' + train['filename'] == filename]['food_type'])
        except Exception as e:
            print(str(e))


images = np.array(images)

train_types_arr = np.concatenate(train_types)
print(train_types_arr)

x_train, x_val, y_train, y_val = train_test_split(images, 
                                                  train_types_arr, 
                                                  test_size=0.2, 
                                                  random_state=1, 
                                                  stratify=train_types_arr)


y_train_dummy = np.array(pd.get_dummies(y_train)) # turn to one-hot
y_val_dummy = np.array(pd.get_dummies(y_val))

  0%|          | 7/11363 [00:00<02:48, 67.55it/s]

11363


100%|██████████| 11363/11363 [02:54<00:00, 65.28it/s]


['beet_salad' 'bibimbap' 'nachos' ... 'lasagna' 'prime_rib' 'filet_mignon']


In [11]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 64

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False, # randomly flip images
    #rescale=1./255,
    zoom_range=[.8, 1],
    fill_mode='reflect') # originally 'nearest'
datagen.fit(x_train)
generator = datagen.flow(x_train, y_train_dummy, batch_size=batch_size)
val_generator = datagen.flow(x_val, y_val_dummy, batch_size=batch_size)


## Model Building

In [12]:
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.optimizers import SGD, RMSprop, Adam


from keras.regularizers import l2

def build_model(model_name="inception_v3"):
    K.clear_session()
    if model_name == "inception_v3":
        from keras.applications.inception_v3 import InceptionV3     
        base_model = InceptionV3(weights='imagenet', 
                                 include_top=False, 
                                 input_tensor=Input(shape=(299, 299, 3)))
    if model_name == "xception_v1":
        from keras.applications.xception import Xception
        base_model = Xception(weights='imagenet', 
                              include_top=False, 
                              input_tensor=Input(shape=(299, 299, 3)))
    if model_name == "mobile_v2":
        from keras.applications.mobilenet_v2 import MobileNetV2
        base_model = MobileNetV2(weights='imagenet', 
                                 include_top=False, 
                                 input_tensor=Input(shape=(299, 299, 3)))
            
    if model_name == "dense_121":
        from keras.applications.densenet import DenseNet121
        base_model = DenseNet121(weights='imagenet', 
                                 include_top=False, 
                                 input_tensor=Input(shape=(299, 299, 3)))
    if model_name == "nas_mobile":
        from keras.applications.nasnet import NASNetMobile
        base_model = NASNetMobile(weights='imagenet', 
                                 include_top=False, 
                                 input_tensor=Input(shape=(299, 299, 3)))
    if model_name == "resnet50":
        from keras.applications.resnet50 import ResNet50
        base_model =ResNet50(weights='imagenet', 
                             include_top=False, 
                             input_tensor=Input(shape=(299, 299, 3)))
    if model_name == "inception_res":
        from keras.applications.inception_resnet_v2 import InceptionResNetV2
        base_model = InceptionResNetV2(weights='imagenet', 
                             include_top=False, 
                             input_tensor=Input(shape=(299, 299, 3)))
    
    x = base_model.output
    x = AveragePooling2D(pool_size=(8, 8))(x) # originally 5,5
    x = Dropout(.4)(x)
    x = Flatten()(x)
    predictions = Dense(101, 
                        init='glorot_uniform', 
                        W_regularizer=l2(.0005), 
                        activation='softmax')(x)
    return base_model, predictions



## Training

In [13]:
def train(model_name, folder_name, restore_path=None):

    base_model, predictions = build_model(model_name)

    model = Model(input=base_model.input, output=predictions)


    for layer in model.layers:
        layer.trainable = True

    model.compile(optimizer=SGD(lr=.01, momentum=.9), loss='categorical_crossentropy', metrics=['accuracy','top_k_categorical_accuracy'])

    if restore_path is not None:
        model.load_weights(restore_path)
        
    if os.path.isdir("training_logs/" + folder_name):
        os.rmdir("training_logs/" + folder_name)
    print("training_logs/" + folder_name)
    os.mkdir("training_logs/" + folder_name)
    os.mkdir("training_logs/" + folder_name + "/logs")
    os.mkdir("training_logs/" + folder_name + "/models")

    from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
    def schedule(epoch):
        if epoch < 12:
            return .01
        elif epoch < 20:
            return .002
        else:
            return .0004
    lr_scheduler = LearningRateScheduler(schedule)
    checkpointer = ModelCheckpoint(filepath='training_logs/' + folder_name + '/models/model.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
    csv_logger = CSVLogger('training_logs/' + folder_name + '/logs/model.log')
    # lr_scheduler = ReduceLROnPlateau(monitor='val_acc',
    #                                          factor=0.2,
    #                                          patience=4,
    #                                          verbose=2,
    #                                          mode='auto',
    #                                          cooldown=0,
    #                                          min_lr=0.0001)

    food_model = model.fit_generator(generator,
                                     validation_data=val_generator,
                                     #nb_val_samples=x_val.shape[0],
                                     validation_steps=x_val.shape[0]/batch_size,
                                     samples_per_epoch=x_train.shape[0],
                                     nb_epoch=15,
                                     verbose=1,
                                     callbacks=[lr_scheduler, csv_logger])
    return food_model, model

## Save Logs And Model

In [14]:
import cloudpickle

def save_logs_and_model(fit_generator_model, model, folder_name):
    res = cloudpickle.dumps(fit_generator_model.history)

    with open("training_logs/" + folder_name + "/logs/history.pkl", "wb") as f:
        f.write(res)

    model.save("training_logs/" + folder_name + "/models/model.h5")


## Run

In [15]:
#model_names = ["inception_v3", "xception_v1", "mobile_v2", "dense_121", "nas_mobile", "resnet50", "inception_res"]
# trained: resnet50, mobile_v2, nas_mobile, inception_res
# can't be trained: xception_v1, dense_121
model_names = ["inception_v3"]
for model_name in model_names:
    folder_name = model_name + "_seed_" + str(seed)
    fit_generator_model, model = train(model_name, 
                                       folder_name, 
                                       restore_path="training_logs/inception_v3_seed_200/models/model.h5")
    save_logs_and_model(fit_generator_model, model, folder_name)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(101, kernel_regularizer=<keras.reg..., activation="softmax", kernel_initializer="glorot_uniform")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """


training_logs/inception_v3_seed_220


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:49: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., verbose=1, validation_data=<keras_pre..., steps_per_epoch=142, epochs=15, callbacks=[<keras.ca..., validation_steps=35)`


Epoch 1/15
142/142 [==============================] - 127s 892ms/step - loss: 0.8203 - acc: 0.8148 - top_k_categorical_accuracy: 0.9478 - val_loss: 1.3530 - val_acc: 0.6987 - val_top_k_categorical_accuracy: 0.9085
Epoch 2/15
142/142 [==============================] - 124s 871ms/step - loss: 0.4479 - acc: 0.8970 - top_k_categorical_accuracy: 0.9815 - val_loss: 0.8693 - val_acc: 0.8090 - val_top_k_categorical_accuracy: 0.9457
Epoch 3/15
142/142 [==============================] - 124s 876ms/step - loss: 0.3015 - acc: 0.9383 - top_k_categorical_accuracy: 0.9899 - val_loss: 0.8524 - val_acc: 0.7995 - val_top_k_categorical_accuracy: 0.9525
Epoch 4/15
142/142 [==============================] - 124s 873ms/step - loss: 0.2983 - acc: 0.9424 - top_k_categorical_accuracy: 0.9911 - val_loss: 0.8141 - val_acc: 0.8189 - val_top_k_categorical_accuracy: 0.9552
Epoch 5/15
142/142 [==============================] - 122s 861ms/step - loss: 0.2063 - acc: 0.9638 - top_k_categorical_accuracy: 0.9945 - val_lo